In [33]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler

In [50]:
# pandas表示件数を設定
pd.set_option('max_columns', 130)
pd.set_option('max_rows', 10)

#### 欠損値を補完する関数

In [27]:
def pdFillNAN(df, strategy = "mean"):
    #Fills empty values with either the mean value of each feature, or an indicated number
    if strategy == "mean":
        return df.fillna(df.mean())
    elif type(strategy) == int:
        return df.fillna(strategy)

In [22]:
def getDummiesInplace(columnList, train, test = None):
    #Takes in a list of column names and one or two pandas dataframes
    #One-hot encodes all indicated columns inplace
    columns = []

    if test is not None:
        df = pd.concat([train,test], axis= 0)
    else:
        df = train

    for columnName in df.columns:
        index = df.columns.get_loc(columnName)
        if columnName in columnList:
            dummies = pd.get_dummies(df.ix[:,index], prefix = columnName, prefix_sep = ".")
            columns.append(dummies)
        else:
            columns.append(df.ix[:,index])
    df = pd.concat(columns, axis = 1)

    if test is not None:
        train = df[:train.shape[0]]
        test = df[train.shape[0]:]
        return train, test
    else:
        train = df
        return train

In [17]:
train = pd.read_csv('data/train.csv') 

In [20]:
train.head(3)

,target,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,16.434108,6.085711,2.866830,11.636387,1.355013,8.571429,3.670350,0.106720,0.148883,18.869283,7.730923,-1.716131e-08,C,0.139412,1.720818,3.393503,0.590122,8.880867,C,A,1.083033,1.010829,7.270147,8.375452,11.326592,0.454546,0,4.012088,7.711453,7.653429,12.707581,2.015505,10.498338,9.848672,0.113561,C,12.171733,8.086643,0.899420,7.277792,G,16.747968,0.037096,1.299638,DI,3.971118,0.529802,10.890984,1.588448,15.858152,1,0.153461,6.363189,18.303925,C,9.314079,15.231789,17.142857,11.784549,F,1,1.614988,B,D,2.230940,7.292418,8.571429,E,3.000000,7.528326,8.861647,0.649820,1.299638,1.707317,0.866426,9.551836,3.321300,0.095678,0.905342,A,0.442252,5.814018,3.517720,0.462019,7.436824,5.454545,8.877414,1.191337,19.470199,8.389237,2.757375,4.374296,1.574039,0.007294,12.579184,E,2.382692,3.930922,B,0.433213,O,NaN,15.634907,2.857144,1.951220,6.592012,5.909091,-6.297423e-07,1.059603,0.803572,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,NaN,1.312910,NaN,6.507647,NaN,11.636386,NaN,NaN,NaN,NaN,NaN,NaN,6.763110,NaN,C,3.056144,NaN,NaN,NaN,NaN,C,A,NaN,NaN,3.615077,NaN,14.579479,NaN,0,NaN,14.305766,NaN,NaN,NaN,NaN,NaN,2.449959,E,NaN,NaN,1.379210,NaN,G,NaN,1.129469,NaN,DY,NaN,NaN,NaN,NaN,NaN,2,2.544736,NaN,NaN,A,NaN,NaN,NaN,12.053353,F,2,NaN,B,D,NaN,NaN,NaN,D,NaN,7.277655,3.430691,NaN,NaN,NaN,NaN,9.848004,NaN,2.678584,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,8.303967,NaN,NaN,NaN,NaN,NaN,NaN,1.505335,NaN,B,1.825361,4.247858,A,NaN,U,G,10.308044,NaN,NaN,10.595357,NaN,NaN,NaN,NaN,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,14.756098,6.384670,2.505589,9.603542,1.984127,5.882353,3.170847,0.244541,0.144258,17.952332,5.245035,-2.785053e-07,E,0.113997,2.244897,5.306122,0.836005,7.499999,NaN,A,1.454082,1.734693,4.043864,7.959184,12.730517,0.259740,0,7.378964,13.077201,6.173469,12.346939,2.926830,8.897561,5.343819,0.126035,C,12.711328,6.836734,0.604504,9.637627,F,15.102041,0.085573,0.765305,AS,4.030613,4.277456,9.105481,2.151361,16.075602,1,0.123643,5.517949,16.377205,A,8.367347,11.040463,5.882353,8.460654,B,3,2.413618,B,B,1.963971,5.918368,11.764705,E,3.333334,10.194433,8.266200,1.530611,1.530613,2.429906,1.071429,8.447465,3.367346,0.111388,0.811447,G,0.271480,5.156559,4.214944,0.309657,5.663265,5.974026,11.588858,0.841837,15.491329,5.879353,3.292788,5.924457,1.668401,0.008275,11.670572,C,1.375753,1.184211,B,3.367348,S,NaN,11.205561,12.941177,3.129253,3.478911,6.233767,-2.792745e-07,2.138728,2.238806,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472


In [19]:
train.drop(labels = 'ID', axis = 1, inplace = True)
train.drop(labels = 'v22', axis = 1, inplace = True)

### ダミー変数作成

In [52]:
categoricalVariables = ['v3', 'v24', 'v30', 'v31', 'v47','v52','v56','v66','v71',
    'v74','v75','v79','v91','v107','v110','v112','v113','v125']

In [39]:
train_dummy = getDummiesInplace(categoricalVariables, train)

In [40]:
train_dummy

,target,v1,v2,v3.A,v3.B,v3.C,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v23,v24.A,v24.B,v24.C,v24.D,v24.E,v25,v26,v27,v28,v29,v30.A,v30.B,v30.C,v30.D,v30.E,v30.F,v30.G,v31.A,v31.B,v31.C,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47.A,v47.B,v47.C,v47.D,v47.E,...,v125.BD,v125.BE,v125.BF,v125.BG,v125.BH,v125.BI,v125.BJ,v125.BK,v125.BL,v125.BM,v125.BN,v125.BO,v125.BP,v125.BQ,v125.BR,v125.BS,v125.BT,v125.BU,v125.BV,v125.BW,v125.BX,v125.BY,v125.BZ,v125.C,v125.CA,v125.CB,v125.CC,v125.CD,v125.CE,v125.CF,v125.CG,v125.CH,v125.CI,v125.CJ,v125.CK,v125.CL,v125.D,v125.E,v125.F,v125.G,v125.H,v125.I,v125.J,v125.K,v125.L,v125.M,v125.N,v125.O,v125.P,v125.Q,v125.R,v125.S,v125.T,v125.U,v125.V,v125.W,v125.X,v125.Y,v125.Z,v126,v127,v128,v129,v130,v131
0,1,1.335739,8.727474,0.0,0.0,1.0,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,16.434108,6.085711,2.866830,11.636387,1.355013,8.571429,3.670350,0.106720,0.148883,18.869283,7.730923,-1.716131e-08,0.0,0.0,1.0,0.0,0.0,0.139412,1.720818,3.393503,0.590122,8.880867,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.083033,1.010829,7.270147,8.375452,11.326592,0.454546,0,4.012088,7.711453,7.653429,12.707581,2.015505,10.498338,9.848672,0.113561,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,1,NaN,NaN,0.0,0.0,1.0,NaN,9.191265,NaN,NaN,2.301630,NaN,1.312910,NaN,6.507647,NaN,11.636386,NaN,NaN,NaN,NaN,NaN,NaN,6.763110,NaN,0.0,0.0,1.0,0.0,0.0,3.056144,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,3.615077,NaN,14.579479,NaN,0,NaN,14.305766,NaN,NaN,NaN,NaN,NaN,2.449959,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.957825,0,NaN,NaN
2,1,0.943877,5.310079,0.0,0.0,1.0,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,14.756098,6.384670,2.505589,9.603542,1.984127,5.882353,3.170847,0.244541,0.144258,17.952332,5.245035,-2.785053e-07,0.0,0.0,0.0,0.0,1.0,0.113997,2.244897,5.306122,0.836005,7.499999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.454082,1.734693,4.043864,7.959184,12.730517,0.259740,0,7.378964,13.077201,6.173469,12.346939,2.926830,8.897561,5.343819,0.126035,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,1,0.797415,8.304757,0.0,0.0,1.0,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,16.347483,9.646653,3.903302,14.094723,1.945044,5.517242,3.610789,1.224114,0.231630,18.376407,7.517125,-4.805344e-07,0.0,0.0,0.0,1.0,0.0,0.148843,1.308269,2.303640,8.926662,8.874521,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.587644,1.666667,8.703550,8.898468,11.302795,0.433735,0,0.287322,11.523045,7.931035,12.935823,1.470878,12.708574,9.670823,0.108387,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,1,NaN,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.050328,NaN,6.320087,NaN,10.991098,NaN,NaN,NaN,NaN,NaN,NaN,6.414567,NaN,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,6.083151,NaN,NaN,NaN,0,NaN,10.138920,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

### 欠損値補完

In [41]:
train_dummy_mean = pdFillNAN(train_dummy)

In [42]:
train_dummy_mean

,target,v1,v2,v3.A,v3.B,v3.C,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v23,v24.A,v24.B,v24.C,v24.D,v24.E,v25,v26,v27,v28,v29,v30.A,v30.B,v30.C,v30.D,v30.E,v30.F,v30.G,v31.A,v31.B,v31.C,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47.A,v47.B,v47.C,v47.D,v47.E,...,v125.BD,v125.BE,v125.BF,v125.BG,v125.BH,v125.BI,v125.BJ,v125.BK,v125.BL,v125.BM,v125.BN,v125.BO,v125.BP,v125.BQ,v125.BR,v125.BS,v125.BT,v125.BU,v125.BV,v125.BW,v125.BX,v125.BY,v125.BZ,v125.C,v125.CA,v125.CB,v125.CC,v125.CD,v125.CE,v125.CF,v125.CG,v125.CH,v125.CI,v125.CJ,v125.CK,v125.CL,v125.D,v125.E,v125.F,v125.G,v125.H,v125.I,v125.J,v125.K,v125.L,v125.M,v125.N,v125.O,v125.P,v125.Q,v125.R,v125.S,v125.T,v125.U,v125.V,v125.W,v125.X,v125.Y,v125.Z,v126,v127,v128,v129,v130,v131
0,1,1.335739,8.727474,0.0,0.0,1.0,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,16.434108,6.085711,2.866830,11.636387,1.355013,8.571429,3.670350,0.106720,0.148883,18.869283,7.730923,-1.716131e-08,0.0,0.0,1.0,0.0,0.0,0.139412,1.720818,3.393503,0.590122,8.880867,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.083033,1.010829,7.270147,8.375452,11.326592,0.454546,0,4.012088,7.711453,7.653429,12.707581,2.015505,10.498338,9.848672,0.113561,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,1,1.630686,7.464411,0.0,0.0,1.0,4.145098,9.191265,2.436402,2.483921,2.301630,9.031859,1.312910,15.447413,6.507647,3.798396,11.636386,2.080911,4.923222,3.832270,0.841045,0.222300,17.773592,6.763110,1.093088e+00,0.0,0.0,1.0,0.0,0.0,3.056144,1.876031,2.743454,5.093328,8.206416,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.622151,2.161633,3.615077,8.122387,14.579479,0.741471,0,1.237184,14.305766,7.182551,12.924965,2.216597,10.795169,9.142231,2.449959,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.672658,3.239542,1.957825,0,1.925763,1.739389
2,1,0.943877,5.310079,0.0,0.0,1.0,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,14.756098,6.384670,2.505589,9.603542,1.984127,5.882353,3.170847,0.244541,0.144258,17.952332,5.245035,-2.785053e-07,0.0,0.0,0.0,0.0,1.0,0.113997,2.244897,5.306122,0.836005,7.499999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.454082,1.734693,4.043864,7.959184,12.730517,0.259740,0,7.378964,13.077201,6.173469,12.346939,2.926830,8.897561,5.343819,0.126035,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,1,0.797415,8.304757,0.0,0.0,1.0,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,16.347483,9.646653,3.903302,14.094723,1.945044,5.517242,3.610789,1.224114,0.231630,18.376407,7.517125,-4.805344e-07,0.0,0.0,0.0,1.0,0.0,0.148843,1.308269,2.303640,8.926662,8.874521,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.587644,1.666667,8.703550,8.898468,11.302795,0.433735,0,0.287322,11.523045,7.931035,12.935823,1.470878,12.708574,9.670823,0.108387,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,1,1.630686,7.464411,0.0,0.0,1.0,4.145098,8.742359,2.436402,2.483921,1.496569,9.031859,1.050328,15.447413,6.320087,3.798396,10.991098,2.080911,4.923222,3.832270,0.841045,0.222300,17.773592,6.4

### 標準化

In [43]:
stdsc = StandardScaler()
train_dummy_mean_std = stdsc.fit_transform(train_dummy_mean)

In [44]:
train_dummy_mean_std

array([[  5.60104792e-01,  -3.62670962e-01,   5.67662358e-01, ...,
         -4.47371660e-01,  -1.35778217e+00,   1.31220083e+00],
       [  5.60104792e-01,   0.00000000e+00,   3.99176601e-15, ...,
         -4.47371660e-01,  -4.44259388e-15,   1.30335904e-15],
       [  5.60104792e-01,  -8.44511796e-01,  -9.68227485e-01, ...,
          2.43755476e+00,  -1.09794261e+00,  -6.60843594e-01],
       ..., 
       [  5.60104792e-01,   0.00000000e+00,   3.99176601e-15, ...,
          2.43755476e+00,  -4.44259388e-15,   1.30335904e-15],
       [  5.60104792e-01,   0.00000000e+00,   3.99176601e-15, ...,
         -4.47371660e-01,  -4.44259388e-15,   1.30335904e-15],
       [  5.60104792e-01,  -1.34305507e-02,   2.10589509e-01, ...,
         -4.47371660e-01,  -1.45480090e-01,  -4.14703602e-01]])

APPEND

In [47]:
# numpy arrayをcsvで保管
#np.savetxt('test.csv', train_dummy_mean_std, delimiter=',')

In [49]:
# csvからnumpy arrayオブジェクトを生成
#ndarr2 = np.loadtxt('test.csv', delimiter=',')

In [11]:
X, y = train.iloc[:, 2:],  train.iloc[:, 1]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)